In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.27-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive # 此时colab中出现drive的文件夹，里面就是你的google drive的根目录文件

In [3]:
import os

path = "/content/drive/XAI/"
os.chdir(path)
os.listdir(path)


['data', 'XAIoutput', 'drive', 'XAIcnn_model.pkl', 'ResNet18.pth', 'CNN.pkl']

In [4]:
filepath='/content/drive/XAI/data/baseset/'
saved_filepath='XAI'

if not os.path.exists(saved_filepath+'output/'):
    os.makedirs(saved_filepath+'output/')

if not os.path.exists(saved_filepath+'output/model/'):
    os.makedirs(saved_filepath+'output/model/')

if not os.path.exists(saved_filepath+'output/loss/'):
    os.makedirs(saved_filepath+'output/loss/')

In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
from tqdm import tqdm
import datetime

import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms,models
import scipy.misc
import cv2

import time

In [6]:
transf=transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [7]:
class MyDataset(Dataset): #创建自己的类：MyDataset,这个类是继承的torch.utils.data.Dataset
    def __init__(self): #初始化一些需要传入的参数
        
        #filepath='XAI\data\baseset\'
       

        # CSV文件读取
        df=pd.read_csv(filepath+'info.csv',encoding="gbk")
        ## 获取所有患者id
        self.image_list = df['ImageDataID'].drop_duplicates().reset_index().rename(columns = {'index':'col',0:'ImageDataID'})
        ## 获取所有患者的年龄和性别信息
        self.demo = df.iloc[self.image_list['col']][['Sex','Age']].reset_index(drop = True)
        self.demo = self.demo.replace('M',0) # 将性别替换为0,1变量
        self.demo = self.demo.replace('F',1)

        self.labels = np.array(self.demo['Age'])
 
    def __getitem__(self, index):   
        id_image = self.image_list['ImageDataID'][index]
        
        label = self.labels[index]


        img=cv2.imread(filepath+id_image+'.png')
        #print(img.size)
        size=(196,160)
        img=cv2.resize(img,size)
        #是否进行transform
        img=transf(img)
        img=img.to(device)
        return torch.tensor(label), img
        
 
    def __len__(self): 
        return len(self.labels)


In [8]:
dataset=MyDataset()

test_num = int(len(dataset)/6)
train_num = len(dataset) - test_num
training_set, test_set = torch.utils.data.random_split(dataset,[train_num, test_num])

train_loader = torch.utils.data.DataLoader(training_set, batch_size=5, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=5, shuffle=True)

loss_function = nn.L1Loss(reduction = 'sum')
train_loss_list = []
test_loss_list = []

In [9]:
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1) 
device=torch.device("cuda")
model = model.to(device)

criterion = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [10]:
num_epochs = 10
start_time = time.time()

for epoch in range(num_epochs):
    """ Training  """
    model.train()

    running_loss = 0.
    running_corrects = 0

    # load a batch data of images
    for i, (labels, inputs) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # get loss value and update the network weights
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(training_set)
    #epoch_acc = running_corrects / len(training_set) * 100.
    print('[Train #{}] Loss: {:.4f} Time: {:.4f}s'.format(epoch, epoch_loss, time.time() - start_time))

    """ Validation"""
    model.eval()

    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0

        for labels, inputs in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(test_set)
        #epoch_acc = running_corrects / len(test_set) * 100.
        print('[Validation #{}] Loss: {:.4f} Time: {:.4f}s'.format(epoch, epoch_loss, time.time() - start_time))


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


[Train #0] Loss: 24.8108 Time: 198.4776s


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


[Validation #0] Loss: 4.0699 Time: 237.8039s
[Train #1] Loss: 4.4805 Time: 265.0309s
[Validation #1] Loss: 4.4554 Time: 265.5576s
[Train #2] Loss: 4.3816 Time: 271.2578s
[Validation #2] Loss: 4.1523 Time: 271.7702s
[Train #3] Loss: 4.2902 Time: 277.4447s
[Validation #3] Loss: 4.0386 Time: 277.9512s
[Train #4] Loss: 4.2639 Time: 283.5803s
[Validation #4] Loss: 4.0345 Time: 284.0948s
[Train #5] Loss: 4.2408 Time: 289.7493s
[Validation #5] Loss: 4.0518 Time: 290.2570s
[Train #6] Loss: 4.2674 Time: 295.8898s
[Validation #6] Loss: 4.6152 Time: 296.3947s
[Train #7] Loss: 4.2485 Time: 302.0696s
[Validation #7] Loss: 4.3857 Time: 302.5875s
[Train #8] Loss: 4.2419 Time: 308.2124s
[Validation #8] Loss: 4.3950 Time: 308.7212s
[Train #9] Loss: 4.3372 Time: 314.3724s
[Validation #9] Loss: 4.0989 Time: 314.8912s


In [ ]:
saved_path='CNN.pkl'
torch.save(model.state_dict(),saved_path)
